In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
df_attacks = pd.read_csv("DATA/attacks.csv",encoding = "ISO-8859-1")
import re
import sys
sys.path.append('../')

import src.cleaning_functions as fc

MIS 3 HIPÓTESIS SON:
- En 2018 hubo más ataques de tiburones tigre en el hemisferio sur que en 1990.
- En México hay más ataques de tiburones blancos que en USA.
- La mayoría de las víctimas que practican surf son mujeres.

Por tanto, para verificar o refutar las 3 hipótesis planteadas, voy a utilizar las siguientes columnas:
- Year
- Country
- Activity
- Sex
- Species

In [2]:
#Voy a proceder a eliminar las columnas del dataframe que, de primeras, no me van a ser de utilidad:

df_attacks.drop(["pdf","href formula","href","Case Number.1","Case Number.2","original order","Unnamed: 22","Unnamed: 23"], axis=1, inplace=True)

In [3]:
df_attacks.shape

(25723, 16)

In [4]:
#Borro filas duplicadas:
borrado=df_attacks.drop_duplicates()

#Comprobamos que, al borrar las filas duplicadas, hemos pasado de 25723 a 6305.
borrado.shape

(6305, 16)

In [6]:
# Borro las filas que sean enteras NaN. En este caso, tras comprobar el shape, veo que solo es una.

borrado.dropna(how="all",inplace=True)
borrado.shape

(6304, 16)

In [13]:
#Remplazo los "nan" por "Unkown" para cada una de las columnas:

borrado["Case Number"].fillna(("Unknown"), inplace=True)
borrado["Date"].fillna(("Unknown"), inplace=True)
borrado["Year"].fillna(("Unknown"), inplace=True)
borrado["Type"].fillna(("Invalid"), inplace=True)
borrado["Country"].fillna(("Unknown"), inplace=True)
borrado["Area"].fillna(("Unknown"), inplace=True)
borrado["Location"].fillna(("Unknown"), inplace=True)
borrado["Activity"].fillna(("Unknown"), inplace=True)
borrado["Name"].fillna(("Unknown"), inplace=True)
borrado["Sex "].fillna(("Unknown"), inplace=True)
#borrado["Age"].fillna(("Unknown"), inplace=True)
borrado["Injury"].fillna(("Unknown"), inplace=True)
borrado["Fatal (Y/N)"].fillna(("Unknown"), inplace=True)
borrado["Time"].fillna(("Unknown"), inplace=True)
borrado["Species "].fillna(("Unknow"),inplace=True)
borrado["Investigator or Source"].fillna(("Unknown"),inplace=True)



/home/lydia/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [16]:
#Compruebo los valores únicos de la columna "Sex":
borrado["Sex "].unique()

#Sustituyo la "M con espacio" por la "M normal". El resto las replazo por "unkonwn" y me quedo únicamente por M y F. Puedo usar replace, no hace falta redex.
df2=borrado.replace({'M ':'M','lli':'Unknown','N':'Unknown','.':'Unknown'})

#Compruebo de nuevo los valores únicos de "Sex" y veo que se han reducido a:"F","M","nan" y "Unknown".
df2["Sex "].unique()


array(['F', 'M', 'Unknown'], dtype=object)

In [17]:
df2.head()
type(df2)

pandas.core.frame.DataFrame

In [18]:
especies=df2["Species "].unique()

In [118]:
df2["Species "]= df2["Species "].str.replace(".*[Ww](hite|HITE).*","White shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Tt](iger|IGER).*","Tiger shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ll](emon|EMON).*","Lemon shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Hh](ammerhead|AMMERHEAD).*","Hammerhead shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Bb](ull|ULL).*","Bull shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Bb](lue|LUE).*","Blue shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](ilvertip|ILVERTIP).*","Silvertip shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Nn](urse|URSE).*","Nurse shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ww](haler|HALER).*","Whaler shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Bb](lacktip|LACKTIP).*","Blacktip shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[MM](ako|AKO).*","Mako shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](and|AMD).*","Sand shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ww](obbegong|OBBEGONG).*","Wobbegong shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Gg](alapagos|ALAPAGOS).*","Galapagos shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Gg](rey|REY).*","Grey shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ll](eopard|EOPARD).*","Leopard shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Zz](ambesi|AMBESI).*","Zambesi shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Bb](lacktail|LACKTAIL).*","Blacktail shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Rr](ed|ED).*","Red shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Dd](usky|USKY).*","Dusky shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Rr](aggedtooth|AGGEDTOOTH).*","Raggedtooth shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](pinner|PINNER).*","Spinner shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Cc](ow|OW).*","Cow shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Pp](orbeagle|ORBEAGLE).*","Porbeagle shark",regex=True)
#df2["Species "]= df2["Species "].str.replace("[^.?!]*Bronze[^.?!]*","Bronze whaler shark",regex=True) #este no sale cuando filtro por "Bronze whaler shark". Ahora me desaparece Bronze directamente...
df2["Species "]= df2["Species "].str.replace(".*[Cc](aribbean|ARIBBEAN).*","Caribbean reef shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](and|AND).*","Sandbar shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](ilky|ILKY).*","Silky shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Zz](ambezi|AMBEZI).*","Zambezi shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Ss](evengill|EVENGILL).*","Sevengill shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Cc](opper|OPPER).*","Copper shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Aa](ngel|NGEL)\s","Angel shark",regex=True) #lo he puesto diferente para que no pille "los ángeles" aún así parece que no ha funcionado.
df2["Species "]= df2["Species "].str.replace(".*[Ss](almon|ALMON).*","Salmon shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Gg](oblin|OBLIN).*","Goblin shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Tt](hresher|HRESHER).*","Thresher shark",regex=True)
df2["Species "]= df2["Species "].str.replace(".*[Dd](ogfish|OGFISH).*","Dogfish shark",regex=True)
df2["Species "]= df2["Species "].str.replace("[^.?!]*involvement[^.?!]*","Involvement not confirmed",regex=True)#no funciona



Saco los valores únicos tras la agruapación y los convierto en lista:

In [119]:
especieslimpio=df2["Species "].unique()
especieslimpio
len(especieslimpio)
listaespecieslimpio=list(especieslimpio)
#print(listaespecieslimpio)

Usando "value.counts", obtengo la frecuencia de cada uno de los valores únicos que he obtenido en el paso anterior y la meto en una variable lista:

In [123]:
frecuencia_especies=list((df2["Species "].value_counts()))

Ahora me creo una lista solo con las frecuencias que son igual a 1. Entiendo que serán los valores que quedan sin armonizar y que quiero meter bajo el nombre genérico "Unkown".

In [125]:
lista2=[]
for i in range(len(frecuencia_especies)):
    if frecuencia_especies[i]==1:
        lista2.append(frecuencia_especies[i])
#print(lista2)
#len(frecuencia_especies)    

Utilizando la función zip con la lista de los valores únicos de la columna Especies y la lista de las frecuencias de las especies, hago un diccionario en el que figura cada uno de los elementos de la columna especie como key y su frecuencia como value.

In [96]:
dicc_ = {}
for especieslimpio,frecuencia_especies in zip(especieslimpio,frecuencia_especies):
    dicc_[especieslimpio] = frecuencia_especies

In [97]:
#print(dicc_ )

El siguiente paso es filtrar aquellos pares cuyo value sea 1. Luego, obtengo las keys y la convierto en lista. Entonces, ya tendría todos los elementos que quiero renombrar con "Unkown".

In [120]:
especiesunkown=dict(filter(lambda x: x[1] == 1, dicc_ .items()))
type(especiesunkown)
#print(especiesunkown)
listaespeciesunkown=list(especiesunkown.keys())
#print((listaespeciesunkown))
#type(listaespeciesunkown)


Construyo una función que recibe dos listas de tamaños diferentes. En este caso, la lista de valores únicos de la columna Especies (la mayor) y otra lista con los valores que quiero sustituir por Unkown (la menor). La función me tiene que devolver la lista de los valores únicos con los elementos de la lista menor reemplazados por "unkown".

In [126]:
def sustituir(s,b):
    for i,p in enumerate(s):
        if p in b:
            b[i]="unkown"
    return b

Con la lista que me devuelve la función, trataría de machacar la columna Especies ya existente. No sé si se podría utilizar .apply o si sería mejor crear una columna nueva y asignarles los valores de la lista resultante de la función para ya luego borrar la columna original de Especies.

In [ ]:
#df["Species "]=df["Species "].apply(sustituir)

In [20]:
#Ahora reemplazo el resto de datos por "unknown":
#df3=df2.replace({'elresto':'Unkown'}) #La duda es: cómo puedo señalar que quiero que me sustituya TOOOODO lo que no he reclasificado yo???

In [21]:
list(df2["Activity"].unique()) # Usar redex. Por ejemplo: donde me encuentre swim sustituye por swimming y así voy creando menos categorías usando replace el metodo de redex por lo otro. str.replace. Las cosas raras que no pueda incluir en ninguna categoría, las pongo como "others"

['Paddling',
 'Standing',
 'Surfing',
 'Free diving',
 'Kite surfing',
 'Swimming',
 'Fishing',
 'Walking',
 'Feeding sharks',
 'Boogie boarding',
 'Scuba diving',
 'Paddle-skiing',
 'Body boarding',
 'Windsurfing',
 'Stand-Up Paddleboarding',
 'Wading',
 'Scuba Diving',
 'Kayak fishing for sharks',
 'Cleaning fish',
 'Snorkeling',
 'Spearfishing',
 'Unknown',
 'Diving',
 'Fishing / Wading',
 '2 boats capsized',
 'Night bathing',
 'Surfing ',
 'Kayaking / Fishing',
 'Kayaking',
 'Body surfing',
 'Swimming, poaching abalone',
 'Canoeing',
 'SUP',
 'Skimboarding',
 'Touching a shark',
 'Fishing ',
 'Attempting to lasso a shark',
 'Photo shoot',
 'Paddle boarding',
 'Kakaying',
 'Washing hands',
 'Grabbing shark for a selfie',
 'Kayak fishing',
 'Tagging sharks',
 'Surf skiing ',
 'Surf fishing',
 'Floating',
 'Cage Diving',
 'Surfng',
 'SUP Foil boarding',
 'Lobstering',
 'Fishing in Alabama Deep Fishing Rodeo',
 'Fishing for squid',
 'Fishing for sharks',
 'Floating in tube',
 'Teasing 

In [22]:
df2["Year"].unique()

array([2018.0, 2017.0, 'Unknown', 2016.0, 2015.0, 2014.0, 2013.0, 2012.0,
       2011.0, 2010.0, 2009.0, 2008.0, 2007.0, 2006.0, 2005.0, 2004.0,
       2003.0, 2002.0, 2001.0, 2000.0, 1999.0, 1998.0, 1997.0, 1996.0,
       1995.0, 1984.0, 1994.0, 1993.0, 1992.0, 1991.0, 1990.0, 1989.0,
       1969.0, 1988.0, 1987.0, 1986.0, 1985.0, 1983.0, 1982.0, 1981.0,
       1980.0, 1979.0, 1978.0, 1977.0, 1976.0, 1975.0, 1974.0, 1973.0,
       1972.0, 1971.0, 1970.0, 1968.0, 1967.0, 1966.0, 1965.0, 1964.0,
       1963.0, 1962.0, 1961.0, 1960.0, 1959.0, 1958.0, 1957.0, 1956.0,
       1955.0, 1954.0, 1953.0, 1952.0, 1951.0, 1950.0, 1949.0, 1948.0,
       1848.0, 1947.0, 1946.0, 1945.0, 1944.0, 1943.0, 1942.0, 1941.0,
       1940.0, 1939.0, 1938.0, 1937.0, 1936.0, 1935.0, 1934.0, 1933.0,
       1932.0, 1931.0, 1930.0, 1929.0, 1928.0, 1927.0, 1926.0, 1925.0,
       1924.0, 1923.0, 1922.0, 1921.0, 1920.0, 1919.0, 1918.0, 1917.0,
       1916.0, 1915.0, 1914.0, 1913.0, 1912.0, 1911.0, 1910.0, 1909.0,
   

Obtengo los valores únicos de la columa Países:

In [23]:
(df2["Country"].unique())

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', 'Unknown', 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO',
       'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 

Creo una función para determinar si cada país de la columna Country pertenece al Hemisferio Norte o al Hemisferio Sur:

In [24]:
def hemisferio (x):
        """
        Esta función devuelve el hemisferio donde se encuentra un país.

        Args:
        x(string):país.
    
        Return: El hemisferio.
    
        """
        
        south = ["Angola", "Argentina", "Australia", "Bolivia", "Botswana", "Brazil", "Burundi", "Chile", "Colombia", "Comoros", "Ecuador", "Fiji" , "Gabon", "Equatorial Guinea", "Indonesia", "Solomon Islands", "Kenya", "Kiribati", "Lesotho", "Madagascar", "Malawi", "Maldives", "Mauritius", "Mozambique" , "Namibia", "Nauru", "New Zealand", "Papua New Guinea", "Paraguay", "Peru", "Republic of the Congo", "Democratic Republic of the Congo", "Rwanda", "Samoa", " Sao Tome and Principe "," Seychelles "," Somalia "," Swaziland "," South Africa "," Tanzania "," East Timor "," Tonga "," Tuvalu "," UgandaUruguay "," Vanuatu "," Zambia " ,"Zimbabwe"]
        south_upper=[]
        for i in range(len(south)):
            (south_upper.append(south[i].upper()))
        

        if x in south_upper:
            return "Hemisferio sur"
        else:
            return "Hemisferio norte"

Ahora usando.apply aplico mi función a la columna Country y veo que efectivamente se crea otra columna llamada Hemisferio.

In [26]:
df2["hemisferio"] = df2["Country"].apply(hemisferio) 
#no me pilla el import de mi hoja de funciones de visual code.

In [27]:
df2.head(16)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,hemisferio
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",Unknown,18h00,White shark,"R. Collier, GSAF",Hemisferio norte
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,Unknown,14h00 -15h00,Unknow,"K.McMurray, TrackingSharks.com",Hemisferio norte
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,Unknown,07h45,Unknow,"K.McMurray, TrackingSharks.com",Hemisferio norte
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,Unknown,Unknown,2 m shark,"B. Myatt, GSAF",Hemisferio sur
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,Unknown,Unknown,Tiger shark,A .Kipper,Hemisferio norte
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",Unknown,Unknown,Unknow,"Daily Telegraph, 6/4/2018",Hemisferio sur
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",Hemisferio sur
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,Unknown,Unknown,Lemon shark,"K. McMurray, TrackingSharks.com",Hemisferio norte
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,Unknown,17h00,Bull shark,"K.McMurray, TrackingSharks.com",Hemisferio norte
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,Unknown,14h00,Unknow,"K. McMurray, Tracking Sharks.com",Hemisferio norte
